In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 数据准备
from sklearn.datasets import load_diabetes
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
diabetes = load_diabetes()
# 查看数据大小
print(diabetes.data.shape)
# 查看数据的keys
print(diabetes.keys())
# 打乱数据集
X,y = shuffle(diabetes.data,diabetes.target,random_state=13)
# 划分训练数据集和测试数据集
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=13)
# 将y转换为列向量
y_train,y_test = y_train.reshape(-1,1),y_test.reshape(-1,1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(442, 10)
dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])
(353, 10)
(89, 10)
(353, 1)
(89, 1)


In [18]:
# 定义线性回归模型主体
def linear_loss(X,y,w,b):
    """
    :param X: 特征
    :param y: 标签
    :param w: 权重
    :param b: 偏置
    :return: 损失函数
    """
    # 训练集的数量 -> (442,10)
    num_train = X.shape[0]
    # 特征数量
    num_feature = X.shape[1]
    # 线性回归预测值
    y_hat = np.dot(X,w) + b
    # 计算损失函数
    loss = np.sum((y_hat-y)**2)/num_train
    # 计算梯度
    dw = np.dot(X.T,(y_hat-y))/num_train # (10,442)*(442,1) -> (10,1) 对应的是矩阵形式的求导公式
    db = np.sum((y_hat-y))/num_train
    print(db.shape)
    return y_hat,loss,db,dw

In [4]:
# 初始化参数
def initialize_parameters(dims):
    """
    :param dims: 特征数量
    :return: 权重和偏置
    """
    w = np.zeros((dims,1))
    b = 0
    return w,b

In [16]:
# 定义模型的训练过程
def linear_train(X,y,learning_rate,epochs=10000):
    """
    :param X: 特征
    :param y: 标签
    :param learning_rate: 学习率
    :param epochs: 迭代次数
    """
    loss_history = []
    # 初始化参数
    w,b=initialize_parameters(X.shape[1])
    # 迭代训练
    for i in range(epochs):
        # 计算当前迭代值，均方损失和梯度
        y_hat,loss,db,dw = linear_loss(X,y,w,b)
        # 更新参数
        w = w - learning_rate*dw
        b = b - learning_rate*db
        print(b)
        # 记录当前迭代损失
        loss_history.append(loss)
        # 打印当前迭代损失
        if i%1000==0:
            print("Epoch: {0}, loss: {1}".format(i,loss))
        # 记录模型参数
        params = {
            "w":w,
            "b":b
        }
        # 记录梯度信息
        grads = {
            "dw":dw,
            "db":db
        }
    return loss_history,params,grads

In [19]:
loss,params,grads = linear_train(X_train,y_train,learning_rate=0.1,epochs=30000)
# print(params)

()
15.165722379603402
Epoch: 0, loss: 28947.98583569405
()
28.814953859619834
()
41.09934530303526
()
52.15538229955786
()
62.105901711652905
()
71.06145656410337
()
79.12154444293078
()
86.3757130529088
()
92.90455521796157
()
98.78060438029493
()
104.0691405485098
()
108.82891564991007
()
113.11280634668499
()
116.96840156966796
()
120.43853129799471
()
123.56174246014344
()
126.37272724428381
()
128.9027085770628
()
131.17978705403635
()
133.22925317662992
()
135.07386836501794
()
136.73411786936853
()
138.22843838965184
()
139.57342293318672
()
140.7840051861801
()
141.873625447885
()
142.8543799711379
()
143.7371553686583
()
144.53174957855265
()
145.24698073311825
()
145.8907851406329
()
146.4703054688445
()
146.99197011000297
()
147.46156460929106
()
147.88429595032397
()
148.26485041201968
()
148.60744563971093
()
148.9158775090818
()
149.1935623036527
()
149.44357467446494
()
149.66868180374996
()
149.87137415218956
()
150.05389313141205
()
150.21825600920488
()
150.3662783241

In [ ]:
# 预测函数
def predict(X,params):
    """
    :param X: 特征
    :param params: 模型参数
    :return: 预测值
    """
    w = params["w"]
    b = params["b"]
    y_hat = np.dot(X,w) + b
    return y_hat

y_pred = predict(X_test,params)
print(y_pred)

回归模型$R^2$系数
衡量的是预测值对于真值的拟合好坏程度。通俗理解，假定$y_i$的方差为1个单位，则$R^2$表示使用该模型之后，$y_i$的残差的方差减少了多少"。比如R方等于0.8，则使用该模型之后残差的方差为原始$y_i$值方差的20%
$$
R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y_i})^2}{\sum_{i=1}^n (y_i - \bar{y})^2}
$$

In [ ]:
# 计算回归模型R^2系数
def r2_score(y,y_hat):
    """
    :param y: 真实值
    :param y_hat: 预测值
    :return: R^2系数
    """
    y_mean = np.mean(y)
    # 总离差平方和
    ss_tot = np.sum((y-y_mean)**2)
    # 回归离差平方和
    ss_res = np.sum((y-y_hat)**2)
    # R^2系数
    r2 = 1 - ss_res/ss_tot
    return r2

print(r2_score(y_test,y_pred))